In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [20]:
from glob import glob
import matplotlib.pyplot as plt
import json

In [7]:
logfiles = sorted(glob('saved/outputs/*'))
print(len(logfiles))

54


In [8]:
pltfiles = sorted(glob('saved/images/plots/*'))
print(len(pltfiles))

54


In [17]:
nTargets = 6
nFolds = 9

In [23]:
meta = {}
for lf, pf in zip(logfiles, pltfiles):
    target = int(lf.split('targ_')[1][0])
    fold = int(lf.split('fold_')[1][0])
#     print(target, fold)
    if target not in meta: meta[target] = [None for _ in range(nFolds)]
    with open(lf) as fl:
        hist = [json.loads(line) for line in fl.read().split('\n') if line]
    ent = dict(log=lf, plot=pf, history=hist)
    meta[target][fold] = ent

In [41]:
def fold_avg_prop(fold, prop):
    agg = 0
    for targ in meta.values():
#         print(targ)
        val = min([ent[prop] for ent in targ[fold]['history']])
        agg += val
#     print(agg)
    return agg / nTargets

In [47]:
PROPS = ['MSE_TEST', 'MAPE_TEST']

print('\t'.join(['metric'] + ['fold-%d' % ii for ii in range(1, 10)]))
print('\t'.join(['MSE'] + ['%.2f' % fold_avg_prop(ii, 'MSE_TEST') for ii in range(nFolds)]))
print('\t'.join(['MAPE'] + ['%.2f' % fold_avg_prop(ii, 'MAPE') for ii in range(nFolds)]))
 

metric	fold-1	fold-2	fold-3	fold-4	fold-5	fold-6	fold-7	fold-8	fold-9
MSE	704.92	180.90	361.23	302.26	256.38	556.54	106.90	300.64	221.11
MAPE	17.73	17.78	18.99	16.96	21.01	18.21	13.42	10.41	15.98
